In [45]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
y = train['label']
X = train.drop('label', axis = 1)
X /= 255

train_idx, val_idx = 29400, 29400 + 6300

X_train, y_train =  X.iloc[:train_idx].values.reshape(-1,28, 28, 1),            y.iloc[:train_idx].values
X_val, y_val =      X.iloc[train_idx:val_idx].values.reshape(-1, 28, 28, 1),    y.iloc[train_idx:val_idx].values
X_test, y_test =    X.iloc[val_idx:].values.reshape(-1, 28, 28, 1),             y.iloc[val_idx:].values

print(X_train.shape)

(29400, 28, 28, 1)


In [28]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from sklearn.metrics import accuracy_score
np.random.seed(1)

model = tf.keras.Sequential([
    tfl.Flatten(input_shape = (28, 28, 1)), 
    tfl.Dense(128, activation= 'relu'), 
    tfl.Dense(64, activation = 'relu'), 
    tfl.Dense(32, activation= 'relu'), 
    tfl.Dense(10, activation = 'softmax')
])

model.compile(optimizer = 'adam', 
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits= False), 
              metrics = ['accuracy'])

model.fit(X_train, y_train, epochs = 50, verbose= 1)

val_pred_probs = model.predict(X_val)
val_pred = tf.argmax(val_pred_probs, axis=1).numpy()
val_accuracy = accuracy_score(y_val, val_pred)
print('Accuracy on the validation set:', round(val_accuracy, 4))

Epoch 1/50


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 684us/step - accuracy: 0.8080 - loss: 0.6260
Epoch 2/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 633us/step - accuracy: 0.9570 - loss: 0.1424
Epoch 3/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 645us/step - accuracy: 0.9715 - loss: 0.0942
Epoch 4/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 630us/step - accuracy: 0.9791 - loss: 0.0667
Epoch 5/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 637us/step - accuracy: 0.9837 - loss: 0.0533
Epoch 6/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 691us/step - accuracy: 0.9862 - loss: 0.0429
Epoch 7/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 627us/step - accuracy: 0.9895 - loss: 0.0305
Epoch 8/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 618us/step - accuracy: 0.9893 - loss: 0.0316
Epoch 9/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 640us/step - accuracy: 0.9922 - loss: 0.0228
Epoch 10/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 628us/step - accuracy: 0.9939 - loss: 0.0181
Epoch 11/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 1s 661us/step - accuracy: 0.9935 - loss: 0.0193
Epoch 12/50
919/919 ━━━━━━━━━━━━━━━━━━━━ 

In [62]:
test_values = test.values.reshape(-1, 28, 28, 1)
test_pred_probs = model.predict(test_values)
test_pred = tf.argmax(test_pred_probs, axis = 1).numpy()
test_df = pd.DataFrame(test_pred, index = test.index, columns = ['Label'])
test_df.index.name = 'ImageId'
test_df.index = test_df.index + 1
#test_df.to_csv('simple_nn_v1.csv')
test_df


875/875 ━━━━━━━━━━━━━━━━━━━━ 0s 252us/step


,Label
ImageId,
1,2
2,0
3,9
4,0
5,3
...,...
27996,9
27997,7
27998,3
